In [1]:
import pandas as pd

In [62]:
import re

In [2]:
import phonenumbers
from phonenumbers import (
    NumberParseException,
    is_possible_number,
    number_type,
    PhoneNumberType
)

In [63]:
def standardizePhone(phone_str: str, default_region: str):
    if pd.isnull(phone_str) or pd.isna(phone_str) or phone_str.strip().lower()=='nan' or phone_str.strip()=='' or phone_str=='0':
        return ''
    if pd.isnull(default_region) or pd.isna(default_region) or default_region.strip().lower()=='nan' or default_region.strip()=='' or default_region=='0':
        return ''
    else:
        try:
            pn = phonenumbers.parse(phone_str, default_region)
            if is_possible_number(pn):
                t = number_type(pn)
                if t == PhoneNumberType.MOBILE:
                    std = f"+{pn.country_code} {pn.national_number}"
                    # tag = "Mobile"
                elif t in (
                    PhoneNumberType.FIXED_LINE,
                    PhoneNumberType.FIXED_LINE_OR_MOBILE,
                    PhoneNumberType.TOLL_FREE
                ):
                    # std = format_number(pn, PhoneNumberFormat.INTERNATIONAL)
                    std = f"+{pn.country_code} {pn.national_number}"
                    # tag = "Landline"
                else:
                    # std = format_number(pn, PhoneNumberFormat.INTERNATIONAL)
                    std = f"+{pn.country_code} {pn.national_number}"
                    # tag = "Other"

                std = std.replace('-', ' ')
                std = re.sub(r'\s+', ' ', std).strip()
                return std

        except NumberParseException:
            pass

        digits = re.sub(r"\D", "", phone_str).lstrip("0")
        default_cc = phonenumbers.country_code_for_region(default_region)
        std = f"+{default_cc} {digits}"
        return std

In [47]:
def standardizeStreet(df, col):
    pattern = r"[^A-Za-z0-9\s\.,\-/']"
    df[col+' std'] = (
        df[col]
        .str.replace('&', 'and')
        .str.replace(pattern, '', regex=True)
        .str.replace(r'\s+', ' ', regex=True)
        .str.replace(r'^[^A-Za-z0-9]+|[^A-Za-z0-9]+$', '', regex=True)
        .str.strip()
    )
    return df[col+' std']

In [59]:
df = pd.read_excel('../5-june-fix/lfa1.xlsx', sheet_name='TELEPHONE 1')

In [60]:
df.columns

Index(['Supplier', 'Country', 'Name 1 (After)', 'Name 2 (Before)',
       'Name 2 (After)', 'Telephone 1 (Before)', 'Telephone 1 (After)',
       'Telephone 2 (Before)', 'Telephone 2 (After)', 'Fax Number (Before)',
       'Fax Number (After)'],
      dtype='object')

In [56]:
df

,Supplier,Address (Before),Street,Street (After),Unnamed: 4,Street std
0,1000012,73400,"PLOT NO:117 & 118,",PLOT NO117 118,False,PLOT NO117 and 118
1,1000117,73715,"PLOT NO.218 & 219,",PLOT NO.218 219,False,PLOT NO.218 and 219
2,1000137,73775,SURVEY NO. 44 & 45,SURVEY NO. 44 45,False,SURVEY NO. 44 and 45
3,1000193,73943,"NO:69/2 & 69/3,",NO69/2 69/3,False,NO69/2 and 69/3
4,1000252,74120,"SITE NO - 25 & 26,",SITE NO - 25 26,False,SITE NO - 25 and 26
...,...,...,...,...,...,...
78,1024596,8584645,"Veerapura Post, 23 & 24,","VEERAPURA POST, 23 24",False,"Veerapura Post, 23 and 24"
79,1025135,9054672,NO 66&67,NO 6667,False,NO 66and67
80,3002527,171435,4 & 5,4 5,False,4 and 5
81,8100163,5918214,V-019130 PLOTNO:39&40 1ST FLOOR CHI,V-019130 PLOTNO3940 1ST FLOOR CHI,False,V-019130 PLOTNO39and40 1ST FLOOR CHI


In [58]:
df.to_csv('../5-june-fix/streetStd.csv')

In [66]:
df['Phone 2 std'] = df.apply(lambda row: standardizePhone(row['Telephone 2 (Before)'], row['Country']), axis=1)

In [67]:
df

,Supplier,Country,Name 1 (After),Name 2 (Before),Name 2 (After),Telephone 1 (Before),Telephone 1 (After),Telephone 2 (Before),Telephone 2 (After),Fax Number (Before),Fax Number (After),Phone 1 std,Phone 2 std
0,1000417,IN,ZELLE BIOTECHNOLOGY PVT LTD,NaN,NaN,22 2945 2152,22 2945 2152,NaN,NaN,NaN,NaN,+91 2229452152,
1,1000521,IN,PARLE ELIZABETH TOOLS PRIVATE LTD,NaN,NaN,0250 6456838,0250 6456838,8433937252,8433937252,NaN,NaN,+91 2506456838,+91 8433937252
2,1000989,IN,GLOBAL TECHNOLOGIES,NaN,NaN,080 23490390,080 23490390,9845303194,9845303194,080 23590380,080 23590380,+91 8023490390,+91 9845303194
3,1001011,IN,KOTHARI DISTRIBUTORS,NaN,NaN,98800 36050,98800 36050,73534 16790,73534 16790,NaN,NaN,+91 9880036050,+91 7353416790
4,1001213,IN,GLOBAL MEDICAL SYSTEMS,NaN,NaN,011 27215547,011 27215547,9810135168,9810135168,011 27115021,011 27115021,+91 1127215547,+91 9810135168
...,...,...,...,...,...,...,...,...,...,...,...,...,...
111,3006506,NL,NETHERLANDS CANCER INSTITUTE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,
112,3006605,US,DATA SCIENCES INTERNATIONAL,(DSI),(DSI),NaN,0016514145700,NaN,0016514145,NaN,NaN,,
113,3006750,NL,Bird & Bird (Netherlands) LLP,NaN,NaN,0651077274,0651077274,NaN,NaN,NaN,NaN,+31 651077274,
114,3006988,NL,Social Venture Group B.V,NaN,NaN,0628772327,0628772327,NaN,NaN,NaN,NaN,+31 628772327,


In [68]:
df.to_csv('../5-june-fix/telephoneStd.csv')